<a href="https://colab.research.google.com/github/hebatallah/constructionBERT/blob/master/classify_severity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# install ktrain on Google Colab
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 170kB/s 
     |████████████████████████████████| 6.7MB 54.1MB/s 
     |████████████████████████████████| 10.0MB 47.4MB/s 
     |████████████████████████████████| 983kB 51.7MB/s 
     |████████████████████████████████| 245kB 57.0MB/s 
     |████████████████████████████████| 1.8MB 45.5MB/s 
     |████████████████████████████████| 501kB 42.8MB/s 
     |████████████████████████████████| 1.0MB 22.0MB/s 
     |████████████████████████████████| 3.7MB 7.7MB/s 
     |████████████████████████████████| 870kB 53.5MB/s 
  Created wheel for ktrain: filename=ktrain-0.11.3-cp36-none-any.whl size=25213801 sha256=12e6b0178d66c50525ca13f64be71e09ce57b96872bced44ed9d27d0a11964d6
  Stored in directory: /root/.cache/pip/wheels/c7/6f/71/4c5097ea14a1e7791668a96c585d7fc81c45f0eb2e7a94b681
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=5f04bd8f6a16f4f5656ef2eed5b5db752dc19f98367bd99010dc450252910747
  Stored in direct

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


In [0]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/Tixierae/WECD/master/classification_data_set.csv

In [0]:
import ktrain
from ktrain import text
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re

# load classification data set
full_set = pd.read_csv('/content/drive/My Drive/constructionBERT/classification_data_set.csv', sep=',')

narrative = full_set['narrative'].tolist()
severity = full_set['severity'].tolist()
injury_type = full_set['injury_type'].tolist()

full_set['cause'] = list(map(lambda tdl: re.search('Cause: (.+?)Fat', str(tdl)).group(1) if re.search('Cause:(.+?)Fat', str(tdl)) != None else 'None', full_set['additional_info'].tolist()))
cause = full_set['cause'].tolist()

full_set = full_set[full_set['injury_type'].notnull()]

lb = LabelEncoder()
full_set['label_severity'] = lb.fit_transform(full_set['severity'])
full_set['severity'] = full_set['severity'].replace('Hospitalized injury', 0)
full_set['severity'] = full_set['severity'].replace('Fatality', 1)
full_set['severity'] = full_set['severity'].replace('Non Hospitalized injury', 2)


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(full_set['narrative'].tolist(), full_set['severity'].tolist(), stratify = full_set['severity'], test_size=0.2, random_state=0) 

target_names = [0,1,2]

In [0]:
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=512, class_names=target_names)
trn = t.preprocess_train(X_train, Y_train)
val = t.preprocess_test(X_test, Y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)
learner.fit_onecycle(2e-5, 5)

preprocessing train...
language: en
train sequence lengths:
	mean : 107
	95percentile : 260
	99percentile : 410


preprocessing test...
language: en
test sequence lengths:
	mean : 108
	95percentile : 267
	99percentile : 477





begin training using onecycle policy with max lr of 2e-05...
Train for 771 steps, validate for 193 steps
Epoch 1/5
210/771 [=======>......................] - ETA: 5:52:03 - loss: 0.8217 - accuracy: 0.6897

In [0]:
matrix = learner.validate(val_data=val)